# Flanker Tutorial: Download Analyses from Flywheel

Welcome! This is an introductory worksheet to explore how we can select and download analysis files from Flywheel. In this example, we will start by walking through each analysis in the *flanker* project, then download the output files. 

**Date modified:** 11/12/2024<br>
**Authors:** Amy Hegarty, Intermountain Neuroimaging Consortium

**Sections:**
1. User Inputs
2. Import Statements
3. Flywheel Login
4. Getting Started
3. Generate Table
4. Download Analysis Files
-----

Before starting...
1. Be sure you have configured your conda environment to view ics managed conda environments and packages. If you haven't get started [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments).

2. Be sure to select the `flywheel` kernel from the list of available kernels. If you don't see the `flywheel` kernel, contact Amy Hegarty <Amy.Hegarty@colorado.edu> or follow the instructions [here](https://inc-documentation.readthedocs.io/en/latest/pl_and_blanca_basics.html#setting-up-conda-environments) to setup a new kernel in a shared conda environment. 

## USER INPUTS
Gather all users defined variables for the worksheet

In [ ]:
user_inputs = {
    "project": "flanker",
    "gear": "bids-feat",  
    "download-path": "/scratch/alpine/ics/flanker"
}


## IMPORT STATEMENTS
Here we will load all packages used in the worksheet. This includes some custom helper functions stored in helper_functions.py

In [ ]:
from pathlib import Path
import os
import flywheel
import pandas as pd
import helper_functions
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('main')

# set default permissions
os.umask(0o002);

## FLYWHEEL LOGIN
Be sure you have first logged into flywheel using the command line interface. Once you have stored your API key, you will not need to log in subsequent times. Follow instructions [here](https://inc-documentation.readthedocs.io/en/latest/cli_basics.html#cli-from-blanca-compute-node). 

In [ ]:
# get flywheel client
fw = flywheel.Client('')

## GETTING STARTED
Let's start by using the flywheel sdk to locate an example analysis in the flanker project, and download a single file. 

In [ ]:
def get_label(cid):
    return fw.get_container(cid).label

In [ ]:
# start by searching for an analysis by name
analysis = fw.analyses.find_one('label=(lower run-01) bids-feat 11/13/24 13:56:15')
# print where the analysis is located in flywheel hieararchy
print(analysis.parents["group"])
print("group: " + get_label(analysis.parents["project"]))
print("project: " + get_label(analysis.parents["subject"]))
print("session: " + get_label(analysis.parents["session"]))
print("analysis: " + get_label(analysis.id))

In [ ]:
# find the first file in the analysis outputs and download it
file = analysis.files[0]

# print file name
print(file.name)

# download
file.download(file.name)

Pause here, do you see the file "bids_tree.html" in you current directory? If so, great! lets clean up our directory and go on to the larger script. If not, pause and ask for help.

In [ ]:
os.remove(file.name)

Now that we now how to locate the analysis objects using flywheel's sdk and download them using flywheel's sdk, lets put it all together. In the following sections, we will use functions already created for us to generate a list of all project analyses and download the analyses to a scratch directory.

## GENERATE TABLE
Use a custom function `get_table` stored in helper_functions.py to walk through all project analyses. Locate analyses which meet our matching critera and ouput their ids in a table. 

In [ ]:
# get summary table
summary = helper_functions.get_table(user_inputs)
# display summary table
summary

In [ ]:
# save summary table for future use
label=user_inputs["project"].lower()+"."+user_inputs["gear"]+".table.csv"
summary.to_csv(label,index=False)
log.info("Analysis Spreadsheet saved: %s", os.path.join(os.getcwd(),label))

## DOWNLOAD FILES
Use the analysis table and download all analysis files to selected path. 

In [ ]:
os.makedirs(user_inputs["download-path"], exist_ok=True)
for index, row in summary.iterrows():
    helper_functions.download_session_analyses_byid(row["analysis.id"],user_inputs["download-path"])